In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.callbacks import *
from keras.optimizers import Adam
from statistics import mean
import os

Using TensorFlow backend.


In [2]:
np.random.seed(7)
ford = pd.read_csv('/Users/aleenaalex/Documents/UCC projects/data/Ford 5 years.csv', header=0, index_col=0)
#ford_sales = pd.read_csv('/Users/aleenaalex/Documents/UCC projects/data/FordSales.csv', header=0, index_col=0)
ftse = pd.read_csv('/Users/aleenaalex/Documents/UCC projects/data/FTSE100.csv', header=0, index_col=0)
dow =  pd.read_csv('/Users/aleenaalex/Documents/UCC projects/data/Dow Jones US AUTO STOCK INDEX-5 Years.csv', header=0, index_col=0)

In [3]:
def normalize(data):
    
    scaler_open = MinMaxScaler()
    scaler_high = MinMaxScaler()
    scaler_low = MinMaxScaler()
    scaler_close = MinMaxScaler()
    scaler_adjclose = MinMaxScaler()
    scaler_volume = MinMaxScaler()
    
    #Open:
    fit = scaler_open.fit(np.array(data['Open']).reshape(-1, 1))
    data['Open'] =scaler_open.transform(np.array(dataframe_ford['Open']).reshape(-1, 1))
    
    #High:
    fit = scaler_high.fit(np.array(data['High']).reshape(-1, 1))
    data['High'] =scaler_high.transform(np.array(data['High']).reshape(-1, 1))
    
    #Low:
    fit = scaler_low.fit(np.array(data['Low']).reshape(-1, 1))
    data['Low'] =scaler_low.transform(np.array(data['Low']).reshape(-1, 1))
    
    #Close:
    fit = scaler_close.fit(np.array(data['Close']).reshape(-1, 1))
    data['Close'] =scaler_close.transform(np.array(data['Close']).reshape(-1, 1))
    
    #AdjClose:
    fit = scaler_adjclose.fit(np.array(data['Adj Close']).reshape(-1, 1))
    data['Adj Close'] =scaler_adjclose.transform(np.array(data['Adj Close']).reshape(-1, 1))
    
    #High:
    fit = scaler_volume.fit(np.array(data['Volume']).reshape(-1, 1))
    data['Volume'] =scaler_volume.transform(np.array(data['Volume']).reshape(-1, 1))
    
    return data,scaler_high

df = ford
dataframe_ford = df
df,scaler = normalize(df)

In [4]:
#convert data: form X and y data:
#print(df.loc[['2015-05-15']])
Xarr = df.rename_axis('ID').values
Xarr = Xarr[0:1258]
Datearr = df.index.values

df1 = df[['High']]
yarr = df1.rename_axis('ID').values

print('Xarr:',Xarr[0:2])
print('yarr:',yarr[0:2])
print('Datearr:',Datearr[0:2])

yarr = np.delete(yarr, 0)
print('yarr:',yarr[0:5])
print('X shape:',Xarr.shape)
print('y shape:',yarr.shape)

Xarr: [[0.95986038 0.96935201 0.97416021 0.98286204 0.96620093 0.04988881]
 [0.97731239 0.98248687 0.98966408 0.99143102 0.97558374 0.05236248]]
yarr: [[0.96935201]
 [0.98248687]]
Datearr: ['2015-05-15' '2015-05-18']
yarr: [0.98248687 0.98161121 0.97548161 0.97460595 0.97022767]
X shape: (1258, 6)
y shape: (1258,)


In [5]:
def build_data(start,pw,window_size=200):
    
    X_train,X_test = Xarr[start:start+window_size],Xarr[start+window_size:start+window_size+pw]
    y_train,y_test = yarr[start:start+window_size],yarr[start+window_size:start+window_size+pw]
    Date_train,Date_test = Datearr[start:start+window_size],Datearr[start+window_size:start+window_size+pw]
    return X_train,X_test,y_train,y_test,Date_train,Date_test

trn_size = len(Xarr)

In [43]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
    
file_path = 'daily_mlp_model_weights.h5'
    
callback_checkpoint = ModelCheckpoint(filepath=file_path,
                                      monitor='val_loss',
                                      save_best_only=True,
                                      save_weights_only=True)
    
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.0009)
    
callbacks = [early_stopping, callback_checkpoint, reduce_lr]
    
tf.keras.backend.clear_session()
    
model_mlp = tf.keras.models.Sequential()
    
model_mlp.add(tf.keras.layers.Reshape((6,),input_shape=(6,)))
    
model_mlp.add(tf.keras.layers.Dense(200,activation='relu'))
    
model_mlp.add(tf.keras.layers.Dense(1,activation='linear'))
    
model_mlp.compile(optimizer='sgd',loss='mse',metrics=['accuracy'])

In [44]:
import time
from itertools import chain
start_time = time.time()

predicted_high = []
actual_high = []
predicted_dates = []
window = 200
model_loss = []
model_val_loss =[]

for trn in range(0,trn_size-window):
    X_train,X_test,y_train,y_test,Date_train,Date_test = build_data(trn,pw=1,window_size = window)
    print(trn)
    
    history = model_mlp.fit(X_train, y_train,verbose = 0, epochs=200,
                        validation_data=(X_test, y_test), callbacks=callbacks, 
                        shuffle=False)
    yhat = model_mlp.predict(X_test, verbose=0)
    yhat = list(chain.from_iterable(yhat)) 
    if len(predicted_high) == 0:
        predicted_high = yhat
        predicted_dates = Date_test.tolist()
        actual_high = y_test.tolist()
        model_loss.append(history.history['loss'][-1])
        model_val_loss.append(history.history['val_loss'][-1])
        
        
    else:
        beg=len(predicted_high)-len(yhat)+1
        yhat_idx = 0
        for pred_idx in range(beg,len(predicted_high)):
            predicted_high[pred_idx]= (predicted_high[pred_idx] + yhat[yhat_idx])/2
            yhat_idx+=1
        predicted_high.append(yhat[-1])
        actual_high.append(y_test[-1])
        predicted_dates.append(Date_test.tolist()[-1])
        model_loss.append(history.history['loss'][-1])
        model_val_loss.append(history.history['val_loss'][-1])
    
    
time_taken = time.time() - start_time

0
Epoch 00064: early stopping
1
Epoch 00011: early stopping
2
Epoch 00011: early stopping
3
Epoch 00011: early stopping
4
5
6
7
8
Epoch 00055: early stopping
9
Epoch 00011: early stopping
10
Epoch 00011: early stopping
11
12
Epoch 00011: early stopping
13
Epoch 00011: early stopping
14
Epoch 00011: early stopping
15
16
17
Epoch 00011: early stopping
18
Epoch 00011: early stopping
19
20
21
22
23
24
Epoch 00011: early stopping
25
26
27
28
Epoch 00011: early stopping
29
Epoch 00011: early stopping
30
Epoch 00011: early stopping
31
Epoch 00036: early stopping
32
Epoch 00042: early stopping
33
Epoch 00060: early stopping
34
Epoch 00073: early stopping
35
Epoch 00011: early stopping
36
Epoch 00011: early stopping
37
Epoch 00011: early stopping
38
Epoch 00011: early stopping
39
Epoch 00011: early stopping
40
Epoch 00011: early stopping
41
42
43
44
Epoch 00057: early stopping
45
Epoch 00011: early stopping
46
Epoch 00040: early stopping
47
Epoch 00031: early stopping
48
Epoch 00067: early stop

389
Epoch 00011: early stopping
390
Epoch 00082: early stopping
391
Epoch 00011: early stopping
392
393
Epoch 00011: early stopping
394
Epoch 00011: early stopping
395
Epoch 00041: early stopping
396
Epoch 00060: early stopping
397
Epoch 00041: early stopping
398
399
400
401
Epoch 00104: early stopping
402
Epoch 00033: early stopping
403
Epoch 00112: early stopping
404
Epoch 00011: early stopping
405
Epoch 00011: early stopping
406
Epoch 00011: early stopping
407
Epoch 00011: early stopping
408
409
Epoch 00140: early stopping
410
Epoch 00011: early stopping
411
Epoch 00021: early stopping
412
Epoch 00073: early stopping
413
Epoch 00086: early stopping
414
Epoch 00011: early stopping
415
Epoch 00011: early stopping
416
Epoch 00011: early stopping
417
Epoch 00011: early stopping
418
Epoch 00011: early stopping
419
420
Epoch 00011: early stopping
421
Epoch 00080: early stopping
422
Epoch 00011: early stopping
423
Epoch 00011: early stopping
424
Epoch 00082: early stopping
425
Epoch 00011:

737
Epoch 00011: early stopping
738
739
740
741
Epoch 00011: early stopping
742
Epoch 00011: early stopping
743
Epoch 00011: early stopping
744
Epoch 00011: early stopping
745
Epoch 00039: early stopping
746
Epoch 00142: early stopping
747
Epoch 00011: early stopping
748
Epoch 00011: early stopping
749
Epoch 00011: early stopping
750
751
Epoch 00011: early stopping
752
Epoch 00011: early stopping
753
Epoch 00011: early stopping
754
Epoch 00011: early stopping
755
756
757
758
759
Epoch 00011: early stopping
760
Epoch 00011: early stopping
761
Epoch 00011: early stopping
762
763
764
Epoch 00015: early stopping
765
Epoch 00082: early stopping
766
Epoch 00011: early stopping
767
768
Epoch 00068: early stopping
769
Epoch 00083: early stopping
770
Epoch 00011: early stopping
771
772
773
Epoch 00011: early stopping
774
Epoch 00011: early stopping
775
Epoch 00011: early stopping
776
777
778
Epoch 00011: early stopping
779
Epoch 00136: early stopping
780
Epoch 00140: early stopping
781
Epoch 00

In [45]:
print("time taken for execution : %s seconds " % (time_taken))
print('Mean Validation loss=', round(mean(model_val_loss),8)) 
print('Mean Model loss=', round(mean(model_loss),8))

time taken for execution : 1177.7024137973785 seconds 
Mean Validation loss= 0.00020772
Mean Model loss= 0.00017003


In [18]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
    
file_path = 'daily_mlp_model_weights.h5'
    
callback_checkpoint = ModelCheckpoint(filepath=file_path,
                                      monitor='val_loss',
                                      save_best_only=True,
                                      save_weights_only=True)
    
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.0009)
    
callbacks = [early_stopping, callback_checkpoint, reduce_lr]
    
tf.keras.backend.clear_session()
    
model_mlp = tf.keras.models.Sequential()
    
model_mlp.add(tf.keras.layers.Reshape((6,),input_shape=(6,)))

model_mlp.add(tf.keras.layers.Dense(190,activation='relu'))
    
model_mlp.add(tf.keras.layers.Dense(1,activation='linear'))
    
model_mlp.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [19]:
import time
from itertools import chain
start_time = time.time()

predicted_high = []
actual_high = []
predicted_dates = []
window = 200
model_loss = []
model_val_loss =[]

for trn in range(0,trn_size-window):
    X_train,X_test,y_train,y_test,Date_train,Date_test = build_data(trn,pw=1,window_size = window)
    print(trn)
    
    history = model_mlp.fit(X_train, y_train,verbose = 0, epochs=200,
                        validation_data=(X_test, y_test), callbacks=callbacks, 
                        shuffle=False)
    yhat = model_mlp.predict(X_test, verbose=0)
    yhat = list(chain.from_iterable(yhat)) 
    if len(predicted_high) == 0:
        predicted_high = yhat
        predicted_dates = Date_test.tolist()
        actual_high = y_test.tolist()
        model_loss.append(history.history['loss'][-1])
        model_val_loss.append(history.history['val_loss'][-1])
        
        
    else:
        beg=len(predicted_high)-len(yhat)+1
        yhat_idx = 0
        for pred_idx in range(beg,len(predicted_high)):
            predicted_high[pred_idx]= (predicted_high[pred_idx] + yhat[yhat_idx])/2
            yhat_idx+=1
        predicted_high.append(yhat[-1])
        actual_high.append(y_test[-1])
        predicted_dates.append(Date_test.tolist()[-1])
        model_loss.append(history.history['loss'][-1])
        model_val_loss.append(history.history['val_loss'][-1])
    
    
time_taken = time.time() - start_time

0
Epoch 00021: early stopping
1
Epoch 00011: early stopping
2
Epoch 00011: early stopping
3
Epoch 00011: early stopping
4
Epoch 00156: early stopping
5
Epoch 00012: early stopping
6
Epoch 00086: early stopping
7
Epoch 00011: early stopping
8
Epoch 00193: early stopping
9
Epoch 00012: early stopping
10
Epoch 00011: early stopping
11
Epoch 00011: early stopping
12
Epoch 00012: early stopping
13
14
Epoch 00047: early stopping
15
Epoch 00011: early stopping
16
Epoch 00016: early stopping
17
Epoch 00011: early stopping
18
Epoch 00013: early stopping
19
Epoch 00011: early stopping
20
Epoch 00013: early stopping
21
Epoch 00011: early stopping
22
Epoch 00011: early stopping
23
Epoch 00013: early stopping
24
Epoch 00015: early stopping
25
Epoch 00015: early stopping
26
Epoch 00013: early stopping
27
Epoch 00013: early stopping
28
Epoch 00016: early stopping
29
Epoch 00013: early stopping
30
Epoch 00019: early stopping
31
Epoch 00011: early stopping
32
Epoch 00012: early stopping
33
Epoch 00012:

Epoch 00015: early stopping
262
Epoch 00013: early stopping
263
Epoch 00020: early stopping
264
Epoch 00012: early stopping
265
Epoch 00020: early stopping
266
Epoch 00012: early stopping
267
Epoch 00012: early stopping
268
Epoch 00015: early stopping
269
Epoch 00030: early stopping
270
Epoch 00014: early stopping
271
Epoch 00011: early stopping
272
Epoch 00012: early stopping
273
Epoch 00012: early stopping
274
Epoch 00028: early stopping
275
Epoch 00037: early stopping
276
Epoch 00025: early stopping
277
Epoch 00021: early stopping
278
Epoch 00011: early stopping
279
Epoch 00023: early stopping
280
Epoch 00012: early stopping
281
Epoch 00055: early stopping
282
Epoch 00027: early stopping
283
Epoch 00014: early stopping
284
Epoch 00014: early stopping
285
Epoch 00015: early stopping
286
Epoch 00015: early stopping
287
Epoch 00015: early stopping
288
Epoch 00039: early stopping
289
Epoch 00018: early stopping
290
Epoch 00014: early stopping
291
Epoch 00017: early stopping
292
Epoch 00

Epoch 00019: early stopping
519
Epoch 00016: early stopping
520
Epoch 00021: early stopping
521
Epoch 00011: early stopping
522
Epoch 00022: early stopping
523
Epoch 00024: early stopping
524
Epoch 00022: early stopping
525
Epoch 00012: early stopping
526
Epoch 00016: early stopping
527
Epoch 00012: early stopping
528
Epoch 00017: early stopping
529
Epoch 00012: early stopping
530
Epoch 00018: early stopping
531
Epoch 00030: early stopping
532
Epoch 00017: early stopping
533
Epoch 00019: early stopping
534
Epoch 00011: early stopping
535
Epoch 00014: early stopping
536
Epoch 00021: early stopping
537
Epoch 00013: early stopping
538
Epoch 00016: early stopping
539
Epoch 00011: early stopping
540
Epoch 00011: early stopping
541
Epoch 00012: early stopping
542
Epoch 00013: early stopping
543
Epoch 00020: early stopping
544
Epoch 00014: early stopping
545
Epoch 00016: early stopping
546
Epoch 00021: early stopping
547
Epoch 00016: early stopping
548
Epoch 00013: early stopping
549
Epoch 00

776
Epoch 00102: early stopping
777
Epoch 00011: early stopping
778
Epoch 00012: early stopping
779
Epoch 00013: early stopping
780
Epoch 00017: early stopping
781
Epoch 00018: early stopping
782
Epoch 00014: early stopping
783
Epoch 00013: early stopping
784
Epoch 00013: early stopping
785
Epoch 00023: early stopping
786
Epoch 00015: early stopping
787
Epoch 00012: early stopping
788
Epoch 00011: early stopping
789
Epoch 00017: early stopping
790
Epoch 00015: early stopping
791
Epoch 00015: early stopping
792
Epoch 00021: early stopping
793
Epoch 00014: early stopping
794
Epoch 00012: early stopping
795
Epoch 00021: early stopping
796
Epoch 00011: early stopping
797
798
Epoch 00012: early stopping
799
Epoch 00023: early stopping
800
Epoch 00031: early stopping
801
Epoch 00018: early stopping
802
Epoch 00012: early stopping
803
Epoch 00161: early stopping
804
Epoch 00012: early stopping
805
Epoch 00020: early stopping
806
Epoch 00021: early stopping
807
Epoch 00012: early stopping
808


Epoch 00083: early stopping
1033
Epoch 00022: early stopping
1034
Epoch 00013: early stopping
1035
Epoch 00015: early stopping
1036
Epoch 00011: early stopping
1037
Epoch 00198: early stopping
1038
Epoch 00011: early stopping
1039
Epoch 00011: early stopping
1040
Epoch 00016: early stopping
1041
Epoch 00016: early stopping
1042
Epoch 00013: early stopping
1043
Epoch 00013: early stopping
1044
Epoch 00011: early stopping
1045
Epoch 00016: early stopping
1046
Epoch 00022: early stopping
1047
Epoch 00011: early stopping
1048
Epoch 00011: early stopping
1049
Epoch 00023: early stopping
1050
Epoch 00027: early stopping
1051
Epoch 00011: early stopping
1052
Epoch 00032: early stopping
1053
Epoch 00011: early stopping
1054
Epoch 00034: early stopping
1055
Epoch 00012: early stopping
1056
Epoch 00013: early stopping
1057
Epoch 00017: early stopping


In [20]:
print("time taken for execution : %s seconds " % (time_taken))
print('Mean Validation loss=', round(mean(model_val_loss),8)) 
print('Mean Model loss=', round(mean(model_loss),8))

time taken for execution : 258.7273552417755 seconds 
Mean Validation loss= 0.00031401
Mean Model loss= 0.00014401
